In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

In [50]:
def symplectic_stabilizer(stabList):
    """
    Input: (str) list of stabilizers in the form "XXI;IXX;XXX"
    Output: ([np.array, np.array]) symplectic representation matrix
    """
    stab = stabList.split(";")
    stab = [i.replace(" ","") for i in stab]
    
    #test
    lenStabs = [len(i) for i in stab]
    eqLens = all(i==lenStabs[0] for i in lenStabs)
    if eqLens==False:
        print("Stab list no equal length")
        return 0
    
    matrixX = np.zeros((len(stab),lenStabs[0]))
    matrixZ = np.zeros((len(stab),lenStabs[0]))

    for indxgen, gen in enumerate(stab):
        for indxop,op in enumerate(gen):
            if op=='X':
                matrixX[indxgen, indxop] = 1
            elif op=='Z':
                matrixZ[indxgen, indxop] = 1
            elif op=='Y':
                matrixX[indxgen, indxop] = 1
                matrixZ[indxgen, indxop] = 1
    
    return [matrixX, matrixZ]

In [3]:
def symplectic_innerProd(op1, op2):
    """
    Input: opi = [binary vector, binary vector]
    Output: (0 or 1) symplectic innerProduct between op1 and op2
    """
    op1 = np.asarray(op1)
    op2 = np.asarray(op2)
    return np.sum(op1[0]*op2[1] + op1[1]*op2[0])%2

In [4]:
def error_syndrome(stabList, error):
    """
    Input:  symplectic matrix of stabilizer
    Output: np.array with error syndrome:
    """
    error_synd = np.zeros(stabList[0].shape[0])
    for i in range(stabList[0].shape[0]):
        error_synd[i] = symplectic_innerProd([stabList[0][i], stabList[1][i]], error)
    
    return error_synd.astype(int)

In [110]:
def classify_errors(stabList):
    """
    Input: stabilizerList
    Output: (dict) with key syndrome and value list of all posible errors in symplectic form
    """
    class_errors = {}
    symplectic_matrix = symplectic_stabilizer(stabList)
    
    numQubits = symplectic_matrix[0].shape[1]
    numStabilizers = symplectic_matrix[0].shape[0]
    
    for error in itertools.product([0,1], repeat=numStabilizers):
        class_errors[tuple(error)] = []
    
    oneError = [np.array(i) for i in itertools.product([0, 1], repeat=numQubits)]
    errors = [[x,y] for x,y in itertools.product(oneError, oneError)]
    
    for error in errors:
        syndrome = error_syndrome(symplectic_matrix, error)
        class_errors[tuple(syndrome)].append(error)

    
    return class_errors
    

In [82]:
# def classify_errors_weight(stabList, weight):
#     """
#     Input: stabilizerList
#     Output: (dict) with key syndrome and value list of all posible errors in symplectic form
#     """
#     class_errors = {}
#     symplectic_matrix = symplectic_stabilizer(stabList)
    
#     numQubits = symplectic_matrix[0].shape[1]
#     numStabilizers = symplectic_matrix[0].shape[0]
    
#     errorW = np.ones(weight)
#     weightErrs = []
#     for i in itertools.permutations(np.append(errorW.astype(int), np.zeros(numQubits-weight).astype(int))):
#         weightErrs.append(i)
    
#     weightErrs = set(weightErrs)
#     weightErrs = [np.asarray(i) for i in weightErrs]
    
    
#     ##NEED TO FIX THERE ARE NO ERRORS OF THE FORM XIIZ
    
#     errorsx = [[x, np.zeros(numQubits).astype(int)] for x in weightErrs]
#     errorsy = [[x,x] for x in weightErrs]
#     errorsz = [[np.zeros(numQubits).astype(int), x] for x in weightErrs]
#     errors = errorsx+errorsy+errorsz
#     for error in errors:
#         syndrome = error_syndrome(symplectic_matrix, error)
#         if tuple(syndrome) in class_errors:
#             class_errors[tuple(syndrome)].append(error)
#         else:
#             class_errors[tuple(syndrome)] = [error]
    
#     return class_errors

In [111]:
def classify_errors_weight(stabList, weight):
    """
    Input: stabilizerList
    Output: (dict) with key syndrome and value list of all posible errors in symplectic form
    """
    class_errors = {}
    symplectic_matrix = symplectic_stabilizer(stabList)
    
    numQubits = symplectic_matrix[0].shape[1]
    numStabilizers = symplectic_matrix[0].shape[0]
    
    for error in itertools.product([0,1], repeat=numStabilizers):
        class_errors[tuple(error)] = []
    
    oneError = [np.array(i) for i in itertools.product([0, 1], repeat=numQubits)]
    errors = [[x,y] for x,y in itertools.product(oneError, oneError) if np.sum(np.maximum(x,y))==weight]
    
    for error in errors:
        syndrome = error_syndrome(symplectic_matrix, error)
        class_errors[tuple(syndrome)].append(error)

    
    return class_errors
    

In [7]:
def minimum_weight_error(errors):
    """
    Input: dictionary with errors in syndromes
    Output: dictionary with lowest weight error in syndromes
    """
    minWeight = {}
    for syndr in errors:
        minW = errors[syndr][0][0].shape[0]
        detected = 0
        for error in errors[syndr]:
            x,y = error
            #weight of error
            w = np.sum(np.maximum(x,y))
            if w<minW:
                minW = w
                detected = [error]
            elif w==minW:
                detected.append(error)
        minWeight[syndr] = detected
    return minWeight      

In [139]:
def symplectic2txt(vector):
    """
    Input: a symplectic vector
    Output: txt operators
    """
    x,z = vector
    operator = ""
    for i in range(x.shape[0]):
        if x[i]==1 and z[i]==0:
            operator+="X"
        elif x[i]==0 and z[i]==1:
            operator+="Z"
        elif x[i]==1 and z[i]==1:
            operator+="Y"
        elif x[i]==0 and z[i]==0:
            operator+="I"
            
    return operator

In [146]:
def syndromes_symplectic2txt(syndrom_dict):
    """
    Input: dictionary of syndromes in symplectic
    Output: dictionary of syndromes in txt
    """
    syndromes = syndrom_dict.copy()
    for i in syndromes:
        if len(syndromes[i])==1:
            syndromes[i] = symplectic2txt(*syndromes[i])
        else:
            newtxt = []
            for j in syndromes[i]:
                newtxt.append(symplectic2txt(j))
            syndromes[i] = newtxt
    return syndromes

In [39]:
def distance_of_code(syndromes):
    """
    Input: dict of syndromes in symplectic form
    Output: distance of code
    """
    commutingOps = syndromes[list(syndromes.keys())[0]]
    
    dist = commutingOps[0][0].shape[0]
    for op in commutingOps:
        x,z = op
        weight = np.sum(np.maximum(x,z))
        
        if weight!=0 and weight<dist:
            dist = weight
    return dist

In [44]:
errors= classify_errors("XZZXI; IXZZX; XIXZZ; ZXIXZ")

In [ ]:
errors2 = classify_errors_weight("XXI; IXX",1)

In [ ]:
dics = minimum_weight_error(errors)
syndromes_symplectic2txt(dics)
    

In [45]:
distance_of_code(errors)

3

In [ ]:
np.maximum(np.array([0,1,2,3]), np.array([3,-1,2,2]))

In [ ]:
g1 = "X Y X 1 X Y X 1 Y Z Y 1 1 1 1 1 Y Z Y 1;"
g2 = "Y Z Y 1 Y Z Y 1 Z X Z 1 1 1 1 1 Z X Z 1;"
g3 = "Y Z Y 1 X Y X 1 X Y X 1 Y Z 1 1 1 1 1 1;"
g4 = "Z X Z 1 Y Z Y 1 Y Z Y 1 Z X Z 1 1 1 1 1;"
g5 = "Z Y Y Z 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1;"
g6 = "X Z Z X 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1;"
g7 = "1 1 1 1 Z Y Y Z 1 1 1 1 1 1 1 1 1 1 1 1;"
g8 = "1 1 1 1 X Z Z X 1 1 1 1 1 1 1 1 1 1 1 1;"
g9 = "1 1 1 1 1 1 1 1 Z Y Y Z 1 1 1 1 1 1 1 1;"
g10= "1 1 1 1 1 1 1 1 X Z Z X 1 1 1 1 1 1 1 1;"
g11= "1 1 1 1 1 1 1 1 1 1 1 1 Z Y Y Z 1 1 1 1;"
g12= "1 1 1 1 1 1 1 1 1 1 1 1 X Z Z X 1 1 1 1;"
g13= "1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 Z Y Y Z;"
g14= "1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 X Z Z X;"
g15= "1 1 Z X 1 1 Z X X X X X X 1 Y Y X 1 Y Y;"
g16= "1 X Z Z X 1 X Z X 1 X Z 1 X Z Z 1 1 1 1"
stabs = g1+g2+g3+g4+g5+g6+g7+g8+g9+g10+g11+g12+g13+g14+g15+g16

In [ ]:
errorsHolography = classify_errors(stabs)

In [ ]:
distance_of_code(errorsHolography)

In [33]:
txt ="XXXXIII; XXIIXXI; XIXIXIX; ZZZZIII; ZZIIZZI; ZIZIZIZ"

In [46]:
errors = classify_errors("XXXXIII; XXIIXXI; XIXIXIX; ZZZZIII; ZZIIZZI; ZIZIZIZ")

In [71]:
distance_of_code(errors)

3

In [149]:
syndromes_symplectic2txt(minimum_weight_error(errors))

{(0, 0, 0, 0, 0, 0): 'IIIIIII',
 (0, 0, 0, 0, 0, 1): 'IIIIIIX',
 (0, 0, 0, 0, 1, 0): 'IIIIIXI',
 (0, 0, 0, 0, 1, 1): 'IIIIXII',
 (0, 0, 0, 1, 0, 0): 'IIIXIII',
 (0, 0, 0, 1, 0, 1): 'IIXIIII',
 (0, 0, 0, 1, 1, 0): 'IXIIIII',
 (0, 0, 0, 1, 1, 1): 'XIIIIII',
 (0, 0, 1, 0, 0, 0): 'IIIIIIZ',
 (0, 0, 1, 0, 0, 1): 'IIIIIIY',
 (0, 0, 1, 0, 1, 0): ['IIIIIXZ', 'IIIIZYI', 'IIIIXIY'],
 (0, 0, 1, 0, 1, 1): ['IIIIIXY', 'IIIIXIZ', 'IIIIYZI'],
 (0, 0, 1, 1, 0, 0): ['IIIXIIZ', 'IIZYIII', 'IIXIIIY'],
 (0, 0, 1, 1, 0, 1): ['IIIXIIY', 'IIXIIIZ', 'IIYZIII'],
 (0, 0, 1, 1, 1, 0): ['IXIIIIZ', 'ZYIIIII', 'XIIIIIY'],
 (0, 0, 1, 1, 1, 1): ['IXIIIIY', 'XIIIIIZ', 'YZIIIII'],
 (0, 1, 0, 0, 0, 0): 'IIIIIZI',
 (0, 1, 0, 0, 0, 1): ['IIIIIZX', 'IIIIZIY', 'IIIIXYI'],
 (0, 1, 0, 0, 1, 0): 'IIIIIYI',
 (0, 1, 0, 0, 1, 1): ['IIIIIYX', 'IIIIXZI', 'IIIIYIZ'],
 (0, 1, 0, 1, 0, 0): ['IIIXIZI', 'IZIYIII', 'IXIIIYI'],
 (0, 1, 0, 1, 0, 1): ['IIXIIZI', 'ZIYIIII', 'XIIIIYI'],
 (0, 1, 0, 1, 1, 0): ['IIIXIYI', 'IXIIIZI', 'IYIZIII'],


In [51]:
minimum_weight_error(errors)[(1, 1, 1, 0, 0, 1)]

[[array([0, 0, 0, 0, 0, 0, 1]), array([0, 1, 0, 0, 0, 0, 1])],
 [array([0, 0, 0, 0, 0, 0, 1]), array([1, 0, 0, 0, 0, 0, 0])],
 [array([1, 1, 0, 0, 0, 0, 0]), array([1, 0, 0, 0, 0, 0, 0])]]

In [52]:
error_syndrome(symplectic_stabilizer(txt), symplectic_stabilizer("XIIIIII"))

array([0, 0, 0, 1, 1, 1])

In [55]:
error_syndrome(symplectic_stabilizer(txt), symplectic_stabilizer("IZIIIII"))

array([1, 1, 0, 0, 0, 0])

In [68]:
error_syndrome(symplectic_stabilizer(txt), symplectic_stabilizer("IYIIIIX"))

array([1, 1, 0, 1, 1, 1])

In [58]:
error_syndrome(symplectic_stabilizer(txt), symplectic_stabilizer("XZIIIII"))

array([1, 1, 0, 1, 1, 1])

In [62]:
symplectic_stabilizer("IYIIIIX")

[array([[0., 1., 0., 0., 0., 0., 1.]]), array([[0., 1., 0., 0., 0., 0., 0.]])]

In [63]:
error_syndrome(symplectic_stabilizer(txt), symplectic_stabilizer("IXIIIIX"))

array([0, 0, 0, 1, 1, 1])

In [67]:
error_syndrome(symplectic_stabilizer(txt), symplectic_stabilizer("IZIIIIX"))

array([1, 1, 0, 0, 0, 1])

In [69]:
symplectic_stabilizer("IYIIIIX")

[array([[0., 1., 0., 0., 0., 0., 1.]]), array([[0., 1., 0., 0., 0., 0., 0.]])]

In [147]:
syndromes_symplectic2txt(classify_errors_weight(txt,1))

{(0, 0, 0, 0, 0, 0): [],
 (0, 0, 0, 0, 0, 1): 'IIIIIIX',
 (0, 0, 0, 0, 1, 0): 'IIIIIXI',
 (0, 0, 0, 0, 1, 1): 'IIIIXII',
 (0, 0, 0, 1, 0, 0): 'IIIXIII',
 (0, 0, 0, 1, 0, 1): 'IIXIIII',
 (0, 0, 0, 1, 1, 0): 'IXIIIII',
 (0, 0, 0, 1, 1, 1): 'XIIIIII',
 (0, 0, 1, 0, 0, 0): 'IIIIIIZ',
 (0, 0, 1, 0, 0, 1): 'IIIIIIY',
 (0, 0, 1, 0, 1, 0): [],
 (0, 0, 1, 0, 1, 1): [],
 (0, 0, 1, 1, 0, 0): [],
 (0, 0, 1, 1, 0, 1): [],
 (0, 0, 1, 1, 1, 0): [],
 (0, 0, 1, 1, 1, 1): [],
 (0, 1, 0, 0, 0, 0): 'IIIIIZI',
 (0, 1, 0, 0, 0, 1): [],
 (0, 1, 0, 0, 1, 0): 'IIIIIYI',
 (0, 1, 0, 0, 1, 1): [],
 (0, 1, 0, 1, 0, 0): [],
 (0, 1, 0, 1, 0, 1): [],
 (0, 1, 0, 1, 1, 0): [],
 (0, 1, 0, 1, 1, 1): [],
 (0, 1, 1, 0, 0, 0): 'IIIIZII',
 (0, 1, 1, 0, 0, 1): [],
 (0, 1, 1, 0, 1, 0): [],
 (0, 1, 1, 0, 1, 1): 'IIIIYII',
 (0, 1, 1, 1, 0, 0): [],
 (0, 1, 1, 1, 0, 1): [],
 (0, 1, 1, 1, 1, 0): [],
 (0, 1, 1, 1, 1, 1): [],
 (1, 0, 0, 0, 0, 0): 'IIIZIII',
 (1, 0, 0, 0, 0, 1): [],
 (1, 0, 0, 0, 1, 0): [],
 (1, 0, 0, 0, 1, 1): [],
 (

In [150]:
import syndrome as sy

In [151]:
sy.syndromes_symplectic2txt(classify_errors_weight(txt,1))

{(0, 0, 0, 0, 0, 0): [],
 (0, 0, 0, 0, 0, 1): 'IIIIIIX',
 (0, 0, 0, 0, 1, 0): 'IIIIIXI',
 (0, 0, 0, 0, 1, 1): 'IIIIXII',
 (0, 0, 0, 1, 0, 0): 'IIIXIII',
 (0, 0, 0, 1, 0, 1): 'IIXIIII',
 (0, 0, 0, 1, 1, 0): 'IXIIIII',
 (0, 0, 0, 1, 1, 1): 'XIIIIII',
 (0, 0, 1, 0, 0, 0): 'IIIIIIZ',
 (0, 0, 1, 0, 0, 1): 'IIIIIIY',
 (0, 0, 1, 0, 1, 0): [],
 (0, 0, 1, 0, 1, 1): [],
 (0, 0, 1, 1, 0, 0): [],
 (0, 0, 1, 1, 0, 1): [],
 (0, 0, 1, 1, 1, 0): [],
 (0, 0, 1, 1, 1, 1): [],
 (0, 1, 0, 0, 0, 0): 'IIIIIZI',
 (0, 1, 0, 0, 0, 1): [],
 (0, 1, 0, 0, 1, 0): 'IIIIIYI',
 (0, 1, 0, 0, 1, 1): [],
 (0, 1, 0, 1, 0, 0): [],
 (0, 1, 0, 1, 0, 1): [],
 (0, 1, 0, 1, 1, 0): [],
 (0, 1, 0, 1, 1, 1): [],
 (0, 1, 1, 0, 0, 0): 'IIIIZII',
 (0, 1, 1, 0, 0, 1): [],
 (0, 1, 1, 0, 1, 0): [],
 (0, 1, 1, 0, 1, 1): 'IIIIYII',
 (0, 1, 1, 1, 0, 0): [],
 (0, 1, 1, 1, 0, 1): [],
 (0, 1, 1, 1, 1, 0): [],
 (0, 1, 1, 1, 1, 1): [],
 (1, 0, 0, 0, 0, 0): 'IIIZIII',
 (1, 0, 0, 0, 0, 1): [],
 (1, 0, 0, 0, 1, 0): [],
 (1, 0, 0, 0, 1, 1): [],
 (